・VGG１９のrelu4_1,relu5_1の特徴量を使用(エンコーダーをVGG19に)

・loss_cx1,loss_sx3、identity_1x1,identity＿2x50にする

・スタイル転送を行う(コンテンツ画像・スタイル画像共にCNN1層)

・identity_lossのときはstyle転送用のCNNを行わない

・reluなしの3*3convを3つをTransformの足算した後に追加する

・スタイル転送時1*1convを1つに変更する

・スタイル転送時1*1に変更する

・デコーダーの出力サイズをx4倍に変更(完成画像の高画質化)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Thu Sep 24 03:27:53 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
import torch.nn as nn
from glob import glob
from tqdm import tqdm
from matplotlib import  pyplot as plt
import numpy as np
import pickle
import torchvision
from PIL import Image
import math

In [ ]:
print(torch.cuda.is_available())
device = torch.device('cuda')

True


In [ ]:
#エンコーダーのセッティング
def making_vgg():
    vgg = nn.Sequential(
    nn.Conv2d(3, 3, (1, 1)),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(3, 64, (3, 3)),
    nn.ReLU(),  # relu1-1
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(64, 64, (3, 3)),
    nn.ReLU(),  # relu1-2
    nn.MaxPool2d((2, 2), (2, 2), (0, 0), ceil_mode=True),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(64, 128, (3, 3)),
    nn.ReLU(),  # relu2-1
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(128, 128, (3, 3)),
    nn.ReLU(),  # relu2-2
    nn.MaxPool2d((2, 2), (2, 2), (0, 0), ceil_mode=True),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(128, 256, (3, 3)),
    nn.ReLU(),  # relu3-1
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),  # relu3-2
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),  # relu3-3
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),  # relu3-4
    nn.MaxPool2d((2, 2), (2, 2), (0, 0), ceil_mode=True),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 512, (3, 3)),
    nn.ReLU(),  # relu4-1, this is the last layer used
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(),  # relu4-2
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(),  # relu4-3
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(),  # relu4-4
    nn.MaxPool2d((2, 2), (2, 2), (0, 0), ceil_mode=True),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(),  # relu5-1
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(),  # relu5-2
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(),  # relu5-3
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU()  # relu5-4
    )
    vgg.load_state_dict(torch.load("/content/drive/My Drive/vgg_normalised.pth"))
    return vgg

In [ ]:
#デコーダーのセッティング
def making_decoder():
    decoder = nn.Sequential(
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 256, (3, 3)),
    nn.ReLU(),
    nn.Upsample(scale_factor=2, mode='nearest'),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 128, (3, 3)),
    nn.ReLU(),
    nn.Upsample(scale_factor=2, mode='nearest'),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(128, 128, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(128, 64, (3, 3)),
    nn.ReLU(),
    nn.Upsample(scale_factor=2, mode='nearest'),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(64, 64, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(64, 32, (3, 3)),
    nn.Upsample(scale_factor=2, mode='nearest'),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(32, 32, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(32, 16, (3, 3)),
    nn.Upsample(scale_factor=2, mode='nearest'),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(16, 16, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(16, 3, (3, 3)),
    )
    decoder.load_state_dict(torch.load("/content/drive/My Drive/SANet++/decoder_iter_525000_63.pth"))
    return decoder

In [ ]:
class SANet(nn.Module):
    def __init__(self, in_planes):
        super(SANet, self).__init__()
        self.f = nn.Conv2d(in_planes, in_planes, (1, 1))
        self.g = nn.Conv2d(in_planes, in_planes, (1, 1))
        self.h = nn.Conv2d(in_planes, in_planes, (1, 1))
        self.sm = nn.Softmax(dim = -1)
        self.out_conv = nn.Conv2d(in_planes, in_planes, (1, 1))
        
        #コンテンツテクスチャ アテンション
        #self.sig = nn.Sigmoid()
        #self.t = nn.Conv2d(in_planes, in_planes, (1, 1))
        #self.glob_ave = nn.AvgPool2d(64)
        #スタイルテクスチャ　アテンション
        #self.l = nn.Conv2d(in_planes, in_planes, (1, 1))
        #self.l_relu = nn.ReLU()

    def new_sqrt(self,feat):
        return torch.pow(feat,0.5)

    def new_var(self,feat):
        mu = torch.mean(feat,dim=2)
        mu = torch.reshape(x=mu,shape=(mu.shape[0],mu.shape[1],1))
        sub = feat - mu
        sub = sub * sub
        var = torch.mean(sub,dim=2)
        shape = feat.shape[2]
        var = (shape/(shape - 1))*var
        return var

    def calc_mean_std(self,feat, eps=1e-5):
        # eps is a small value added to the variance to avoid divide-by-zero.
        size = feat.size()
        assert (len(size) == 4)
        N, C = size[:2]
        feat_var = self.new_var(feat.view(N, C, -1)) + eps
        #feat_var = feat.view(N, C, -1).var(dim=2) + eps
        feat_std = self.new_sqrt(feat_var).view(N, C, 1, 1)
        #feat_std = feat_var.sqrt().view(N, C, 1, 1)
        feat_mean = feat.view(N, C, -1).mean(dim=2).view(N, C, 1, 1)
        return feat_mean, feat_std

    def mean_variance_norm(self,feat):
        size = feat.size()
        mean, std = self.calc_mean_std(feat)
        normalized_feat = (feat - mean.expand(size)) / std.expand(size)
        return normalized_feat

    def forward(self, content, style):
        F = self.f(self.mean_variance_norm(content))
        G = self.g(self.mean_variance_norm(style))
        
        #スタイルに関する必要なテクスチャ をアテンションする
        #self.glob_ave = nn.AvgPool2d(style.shape[2])
        #style_glob_ave_output = self.glob_ave(style)
        #l_output = self.l(style_glob_ave_output)
        #l_relu_output = self.l_relu(l_output)
        #l_sm_output = self.sig(l_relu_output)
        #style_texture = style*l_sm_output
        
        H = self.h(style)
        b, c, h, w = F.size()
        F = F.view(b, c, w * h).permute(0, 2, 1)
        b, c, h, w = G.size()
        G = G.view(b, c, w * h)
        #F = F[0]
        #G = G[0]
        S = torch.bmm(F, G)
        S = self.sm(S)
        b, c, h, w = H.size()
        H = H.view(b, c, w * h)
        S = S.permute(0, 2, 1)
        O = torch.bmm(H, S)
        b, c, h, w = content.size()
        O = O.view(b, c, h, w)
        O = self.out_conv(O)
        #コンテンツ に関する必要なテクスチャ をアテンションする
        #self.glob_ave = nn.AvgPool2d(content.shape[2])
        #glob_ave_output = self.glob_ave(content)
        #t_output = self.t(glob_ave_output)
        #t_sm_output = self.sig(t_output)
        O = O + content
        return O

In [ ]:
class VGG_Attention_Content(nn.Module):
    def __init__(self,dim):
        super().__init__()
        #スタイル転送に対するCNN
        self.cnn = nn.Conv2d(dim,dim,(1,1))
    def forward(self,feats):
        #スタイル転送に対するCNN
        output = self.cnn(feats)
        return output

class VGG_Attention_Style(nn.Module):
    def __init__(self,dim):
        super().__init__()
        self.cnn = nn.Conv2d(dim,dim,(1,1))
    def forward(self,feats):
        output = self.cnn(feats)
        return output

In [ ]:
#エンコードされて受け取った特徴マップをスタイル転送用に学習し直す(エンコーダーの質をあげるため)
vgg_stylized_conv_5_content = VGG_Attention_Content(dim=512).to(device)
vgg_stylized_conv_4_content = VGG_Attention_Content(dim=512).to(device)
vgg_stylized_conv_3_content = VGG_Attention_Content(dim=256).to(device)
vgg_stylized_conv_2_content = VGG_Attention_Content(dim=128).to(device)
vgg_stylized_conv_1_content = VGG_Attention_Content(dim=64).to(device)

vgg_stylized_conv_5_style = VGG_Attention_Style(dim=512).to(device)
vgg_stylized_conv_4_style = VGG_Attention_Style(dim=512).to(device)
vgg_stylized_conv_3_style = VGG_Attention_Style(dim=256).to(device)
vgg_stylized_conv_2_style = VGG_Attention_Style(dim=128).to(device)
vgg_stylized_conv_1_style = VGG_Attention_Style(dim=64).to(device)

In [ ]:
class VGG_Stylized_Content(nn.Module):
    def __init__(self):
        super(VGG_Stylized_Content, self).__init__()
        self.vgg_stylized_conv_5 = vgg_stylized_conv_5_content
        self.vgg_stylized_conv_4 = vgg_stylized_conv_4_content
        self.vgg_stylized_conv_3 = vgg_stylized_conv_3_content
        self.vgg_stylized_conv_2 = vgg_stylized_conv_2_content
        self.vgg_stylized_conv_1 = vgg_stylized_conv_1_content

    def forward(self,inputs):
        #sa_output_3 = self.conv3_1(self.conv_pad_3_1(self.ave_pool_3_1(self.sanet3_1(content_feats[-3], style_feats[-3]))))
        results = []
        for i in range(5):
            func = getattr(self, 'vgg_stylized_conv_{:d}'.format(i+1))
            results.append(func(inputs[i]))
        return results
        #return self.merge_conv(self.merge_conv_pad(sa_output_4 + sa_output_5))

class VGG_Stylized_Style(nn.Module):
    def __init__(self):
        super(VGG_Stylized_Style, self).__init__()
        self.vgg_stylized_conv_5_style = vgg_stylized_conv_5_style
        self.vgg_stylized_conv_4_style = vgg_stylized_conv_4_style
        self.vgg_stylized_conv_3_style = vgg_stylized_conv_3_style
        self.vgg_stylized_conv_2_style = vgg_stylized_conv_2_style
        self.vgg_stylized_conv_1_style = vgg_stylized_conv_1_style

    def forward(self,inputs):
        #sa_output_3 = self.conv3_1(self.conv_pad_3_1(self.ave_pool_3_1(self.sanet3_1(content_feats[-3], style_feats[-3]))))
        results = []
        for i in range(5):
            func = getattr(self, 'vgg_stylized_conv_{:d}_style'.format(i+1))
            results.append(func(inputs[i]))
        return results
        #return self.merge_conv(self.merge_conv_pad(sa_output_4 + sa_output_5))

In [ ]:
def calc_mean_std(feat, eps=1e-5):
    # eps is a small value added to the variance to avoid divide-by-zero.
    size = feat.size()
    assert (len(size) == 4)
    N, C = size[:2]
    feat_var = feat.view(N, C, -1).var(dim=2) + eps
    feat_std = feat_var.sqrt().view(N, C, 1, 1)
    feat_mean = feat.view(N, C, -1).mean(dim=2).view(N, C, 1, 1)
    return feat_mean, feat_std
def normal(feat, eps=1e-5):
    feat_mean, feat_std= calc_mean_std(feat, eps)
    normalized=(feat-feat_mean)/feat_std
    return normalized  

In [ ]:
class TransformAttention(nn.Module):
  def __init__(self,channel):
    super().__init__()
    self.padding = nn.ReflectionPad2d(1)
    self.conv = nn.Conv2d(channel,channel,kernel_size=(3,3))
    self.sigmoid = nn.Sigmoid()
  def forward(self,input):
    normal_input = normal(input)
    padding_output = self.padding(normal_input)
    conv_output = self.conv(padding_output)
    sigmoid_output = self.sigmoid(conv_output)
    return sigmoid_output

In [ ]:
class Transform(nn.Module):
    def __init__(self, in_planes_4,in_planes_5):
        super(Transform, self).__init__()
        self.sanet4_1 = SANet(in_planes = in_planes_4)
        self.sanet5_1 = SANet(in_planes = in_planes_5)
        self.upsample5_1 = nn.Upsample(scale_factor=2, mode='nearest')

        self.merge_conv_pad = nn.ReflectionPad2d((1, 1, 1, 1))
        self.merge_conv1 = nn.Conv2d(in_planes_5, 512, (3, 3))
        self.merge_conv2 = nn.Conv2d(512, 512, (3, 3))
        self.merge_conv3 = nn.Conv2d(512, 512, (3, 3))

    def forward(self,content4_1, style4_1, content5_1, style5_1):
        sa_output_4 = self.sanet4_1(content4_1, style4_1)
        sa_output_5 = self.upsample5_1(self.sanet5_1(content5_1, style5_1))
        merge_conv_1_output = self.merge_conv1(self.merge_conv_pad(sa_output_4+sa_output_5))
        merge_conv_2_output = self.merge_conv2(self.merge_conv_pad(merge_conv_1_output))
        merge_conv_3_output = self.merge_conv3(self.merge_conv_pad(merge_conv_2_output))
        return merge_conv_3_output

In [ ]:
def making_vgg_stylized():
    #vgg_stylized = VGG_Stylized()
    #vgg_stylized.load_state_dict(torch.load("/content/drive/My Drive/SANetPytorch/vgg_stylized_iter_100000_7.pth"))
    vgg_stylized_content = VGG_Stylized_Content()
    vgg_stylized_style = VGG_Stylized_Style()
    vgg_stylized_content.load_state_dict(torch.load("/content/drive/My Drive/SANet++/vgg_stylized_content_iter_525000_63.pth"))
    vgg_stylized_style.load_state_dict(torch.load("/content/drive/My Drive/SANet++/vgg_stylized_style_iter_525000_63.pth"))
    return vgg_stylized_content,vgg_stylized_style
    #return vgg_stylized

In [ ]:
def making_sanet():
    sanet = Transform(512,512)
    sanet.load_state_dict(torch.load("/content/drive/My Drive/SANet++/sa_module_iter_525000_63.pth"))
    return sanet

In [ ]:
vgg = making_vgg()
vgg.to(device)
vgg_stylized_content,vgg_stylized_style = making_vgg_stylized()
sa_module = making_sanet()
decoder = making_decoder()
decoder.eval().to(device)

Sequential(
  (0): ReflectionPad2d((1, 1, 1, 1))
  (1): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1))
  (2): ReLU()
  (3): Upsample(scale_factor=2.0, mode=nearest)
  (4): ReflectionPad2d((1, 1, 1, 1))
  (5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
  (6): ReLU()
  (7): ReflectionPad2d((1, 1, 1, 1))
  (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
  (9): ReLU()
  (10): ReflectionPad2d((1, 1, 1, 1))
  (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
  (12): ReLU()
  (13): ReflectionPad2d((1, 1, 1, 1))
  (14): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1))
  (15): ReLU()
  (16): Upsample(scale_factor=2.0, mode=nearest)
  (17): ReflectionPad2d((1, 1, 1, 1))
  (18): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (19): ReLU()
  (20): ReflectionPad2d((1, 1, 1, 1))
  (21): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
  (22): ReLU()
  (23): Upsample(scale_factor=2.0, mode=nearest)
  (24): ReflectionPad2d((1, 1, 1, 1))
  (25): Conv2d(64, 64, 

In [ ]:
def big_images(images):
  rcan_output = rcan(images)
  return rcan_output

def small_images(images):
  output = torch.nn.functional.interpolate(images,scale_factor=0.25,mode="bicubic")
  return output

In [ ]:
class Net(nn.Module):
    def __init__(self,encoder , decoder):
        super(Net, self).__init__()
        self.encoder = encoder
        enc_layers = list(encoder.children())
        self.enc_1 = nn.Sequential(*enc_layers[:4])  # input -> relu1_1
        self.enc_2 = nn.Sequential(*enc_layers[4:11])  # relu1_1 -> relu2_1
        self.enc_3 = nn.Sequential(*enc_layers[11:18])  # relu2_1 -> relu3_1
        self.enc_4 = nn.Sequential(*enc_layers[18:31])  # relu3_1 -> relu4_1
        self.enc_5 = nn.Sequential(*enc_layers[31:44])  # relu4_1 -> relu5_1
        #transform
        self.sa_module = sa_module
        self.upsampling = nn.Upsample(32)
        self.vgg_stylized_content = vgg_stylized_content
        self.vgg_stylized_style = vgg_stylized_style
        self.decoder = decoder
        self.mse_loss = nn.MSELoss()
        self.variation_loss = nn.L1Loss()
        # fix the encoder
        for name in ['enc_1', 'enc_2', 'enc_3', 'enc_4', 'enc_5']:
            for param in getattr(self, name).parameters():
                param.requires_grad = False

    # extract relu1_1, relu2_1, relu3_1, relu4_1, relu5_1 from input image
    def encode_with_intermediate(self, input):
        results = [input]
        for i in range(5):
            func = getattr(self, 'enc_{:d}'.format(i + 1))
            results.append(func(results[-1]))
        return results[1:]

    def calc_content_loss(self, input, target):
        assert (input.size() == target.size())
        #assert (target.requires_grad is False)
        return self.mse_loss(input, target)

    def calc_style_loss(self, input, target):
        assert (input.size() == target.size())
        #assert (target.requires_grad is False)
        input_mean, input_std = calc_mean_std(input)
        target_mean, target_std = calc_mean_std(target)
        return self.mse_loss(input_mean, target_mean) + \
               self.mse_loss(input_std, target_std)
    
    def compute_total_variation_loss_l1(self, inputs):
        h = inputs.shape[2]
        w = inputs.shape[3]
        h1 = inputs[:, :, 0:h-1, :]
        h2 = inputs[:, :, 1:h, :]
        w1 = inputs[:, :, :, 0:w-1]
        w2 = inputs[:, :, :, 1:w]
        return self.variation_loss(h1, h2)+self.variation_loss(w1, w2)

    def generate_image(self,content,style):
        style_feats = self.encode_with_intermediate(style)
        content_feats = self.encode_with_intermediate(content)
        new_style_feats = self.vgg_stylized_style(style_feats)
        new_content_feats = self.vgg_stylized_content(content_feats)
        sanet_output = self.sa_module(new_content_feats[-2],new_style_feats[-2],new_content_feats[-1],new_style_feats[-1])
        decoder_input = sanet_output
        Ics = self.decoder(decoder_input)
        return Ics
    
    def forward(self, content, style):
        style_feats = self.encode_with_intermediate(style)
        content_feats = self.encode_with_intermediate(content)
        new_style_feats = self.vgg_stylized_style(style_feats)
        new_content_feats = self.vgg_stylized_content(content_feats)
        #スタイル変換用
        sanet_output = self.sa_module(new_content_feats[-2],new_style_feats[-2],new_content_feats[-1],new_style_feats[-1])
        decoder_input = sanet_output
        Ics = self.decoder(decoder_input)
        Ics = small_images(Ics)
        Ics_feats = self.encode_with_intermediate(Ics)
        # Variation loss
        loss_v = self.compute_total_variation_loss_l1(Ics)
        # Content loss
        loss_c = self.calc_content_loss(normal(Ics_feats[0]), normal(content_feats[0]))+self.calc_content_loss(normal(Ics_feats[1]), normal(content_feats[1]))+self.calc_content_loss(normal(Ics_feats[2]), normal(content_feats[2]))+self.calc_content_loss(normal(Ics_feats[3]), normal(content_feats[3]))+self.calc_content_loss(normal(Ics_feats[4]), normal(content_feats[4]))
        # Style loss
        loss_s = self.calc_style_loss(Ics_feats[0], style_feats[0])+self.calc_style_loss(Ics_feats[1], style_feats[1])+self.calc_style_loss(Ics_feats[2], style_feats[2])+self.calc_style_loss(Ics_feats[3], style_feats[3])+self.calc_style_loss(Ics_feats[4], style_feats[4])

        #new_style_feats_content = self.vgg_stylized_content(style_feats)
        #new_content_feats_style = self.vgg_stylized_style(content_feats)
        #Identity losses lambda 1
        Icc = self.decoder(self.sa_module(content_feats[-2],content_feats[-2],content_feats[-1],content_feats[-1]))
        Iss = self.decoder(self.sa_module(style_feats[-2],style_feats[-2],style_feats[-1],style_feats[-1])) 

        Icc = small_images(Icc)
        Iss = small_images(Iss)
        
        loss_lambda1 = self.calc_content_loss(Icc,content)+self.calc_content_loss(Iss,style)
        
        #Identity losses lambda 2
        Icc_feats=self.encode_with_intermediate(Icc)
        Iss_feats=self.encode_with_intermediate(Iss)
        loss_lambda2 = self.calc_content_loss(Icc_feats[0], content_feats[0])+self.calc_content_loss(Iss_feats[0], style_feats[0])
        for i in range(1, 5):
            loss_lambda2 += self.calc_content_loss(Icc_feats[i], content_feats[i])+self.calc_content_loss(Iss_feats[i], style_feats[i])
        return loss_c, loss_s, loss_lambda1, loss_lambda2,loss_v

### 訓練

In [ ]:
network = Net(vgg,decoder)
network.train()
network.to(device)

Net(
  (encoder): Sequential(
    (0): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
    (1): ReflectionPad2d((1, 1, 1, 1))
    (2): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): ReflectionPad2d((1, 1, 1, 1))
    (5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (6): ReLU()
    (7): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0), dilation=1, ceil_mode=True)
    (8): ReflectionPad2d((1, 1, 1, 1))
    (9): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (10): ReLU()
    (11): ReflectionPad2d((1, 1, 1, 1))
    (12): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0), dilation=1, ceil_mode=True)
    (15): ReflectionPad2d((1, 1, 1, 1))
    (16): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
    (17): ReLU()
    (18): ReflectionPad2d((1, 1, 1, 1))
    (19): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
    (20): ReLU()
    (21): Reflec

In [ ]:
optimizer = torch.optim.Adam([{'params': network.decoder.parameters()},{'params': network.sa_module.parameters()},{'params': network.vgg_stylized_content.parameters()},{'params': network.vgg_stylized_style.parameters()}], lr=1e-4)

In [ ]:
def adjust_learning_rate(optimizer, iteration_count):
    """Imitating the original implementation"""
    lr = 1e-4 / (1.0 + 5e-5 * iteration_count)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
MAX_ITER = 1000000
SAVE_MODEL_INTERVAL = 5000
BATCH_SIZE = 4

In [ ]:
def getting_coco_dataset(i):
  j = i * 1000 + 1000
  files = glob("/content/drive/My Drive/MSCOCO_torch_training_256_new/*_{}.pickle".format(j))
  file = files[0]
  print(file)
  with open(file=file,mode="rb") as f:
    image_arrays = pickle.load(file=f)
  return image_arrays

def getting_wikiart_dataset(i):
  j = i * 1000 + 1000
  files = glob("/content/drive/My Drive/WIKIART_torch_training/*_{}.pickle".format(j))
  file = files[0]
  print(file)
  with open(file=file,mode="rb") as f:
    image_arrays = pickle.load(file=f)
  return image_arrays

In [ ]:
loss_array = []
loss_c_array = []
loss_s_array = []
loss_identity_1_array = []
loss_identity_2_array = []
loss_v_array = []

In [ ]:
def test_image(content,style):
    coco_file = content
    coco=Image.open(coco_file)
    coco = coco.convert('RGB')
    _DEFAULT_SIZE = 512
    if coco.width > coco.height:
        p = torchvision.transforms.Compose([
                                        torchvision.transforms.Resize((int(_DEFAULT_SIZE*(coco.width/coco.height)),_DEFAULT_SIZE)),
                                        torchvision.transforms.RandomCrop(512)
                                        ])
    else:
        p = torchvision.transforms.Compose([
                                        torchvision.transforms.Resize((_DEFAULT_SIZE,int(_DEFAULT_SIZE*(coco.height/coco.width)))),
                                        torchvision.transforms.RandomCrop(512)
                                        ])
    coco_tensor = torchvision.transforms.functional.to_tensor(p(coco))
    coco_tensor = torch.unsqueeze(coco_tensor,dim=0).to(device)
    wiki_file = style
    wiki=Image.open(wiki_file)
    wiki = wiki.convert('RGB')
    if wiki.width > wiki.height:
        p = torchvision.transforms.Compose([
                                        torchvision.transforms.Resize((int(_DEFAULT_SIZE*(wiki.width/wiki.height)),_DEFAULT_SIZE)),
                                        torchvision.transforms.RandomCrop(512)
                                        ])
    else:
        p = torchvision.transforms.Compose([
                                        torchvision.transforms.Resize((_DEFAULT_SIZE,int(_DEFAULT_SIZE*(wiki.height/wiki.width)))),
                                        torchvision.transforms.RandomCrop(512)
                                        ])
    wiki_tensor = torchvision.transforms.functional.to_tensor(p(wiki))
    wiki_tensor = torch.unsqueeze(wiki_tensor,dim=0).to(device)
    decoder_output = network.generate_image(coco_tensor,wiki_tensor)
    image = decoder_output[0].cpu()
    image = image.detach().numpy()
    generated_imge = image
    image = np.transpose(image, (1,2,0))
    plt.imshow(image)
    plt.show()

In [ ]:
test_image("/content/brad_pitt.jpg","/content/023.jpg")
test_image("/content/brad_pitt.jpg","/content/la_muse.jpg")
test_image("/content/brad_pitt.jpg","/content/impronte_d_artista.jpg")
test_image("/content/brad_pitt.jpg","/content/fire.jpg")
test_image("/content/brad_pitt.jpg","/content/goph.jpg")
test_image("/content/brad_pitt.jpg","/content/in1.jpg")
test_image("/content/brad_pitt.jpg","/content/in2.jpg")
test_image("/content/brad_pitt.jpg","/content/in3.jpg")
test_image("/content/brad_pitt.jpg","/content/in4.jpg")

test_image("/content/cat.jpg","/content/023.jpg")
test_image("/content/cat.jpg","/content/la_muse.jpg")
test_image("/content/cat.jpg","/content/impronte_d_artista.jpg")
test_image("/content/cat.jpg","/content/fire2.jpg")
test_image("/content/cat.jpg","/content/goph.jpg")
test_image("/content/cat.jpg","/content/in1.jpg")
test_image("/content/cat.jpg","/content/in2.jpg")
test_image("/content/cat.jpg","/content/in3.jpg")
test_image("/content/cat.jpg","/content/in4.jpg")

In [ ]:
for i in tqdm(range(525000,MAX_ITER)):
  #1000/BATCH_SIZEに1度呼び出して、画像データを入れ替える
  if i % int(1000/BATCH_SIZE) == 0:
    if i != 0:
      count = int(int((i)*BATCH_SIZE/1000) % 80)
      #Tensor型配列(要素数1000)が入っている
      content_images_datasets = getting_coco_dataset(count)
      style_images_datasets = getting_wikiart_dataset(count)
    elif i == 0:
      count = 0
      #Tensor型配列(要素数1000)が入っている
      content_images_datasets = getting_coco_dataset(count)
      style_images_datasets = getting_wikiart_dataset(count)
  now_count = i % int(1000/BATCH_SIZE)
  content_images = content_images_datasets[now_count:now_count+BATCH_SIZE]
  style_images = style_images_datasets[now_count:now_count+BATCH_SIZE]
  content_images = torch.cat(content_images).reshape(len(content_images), *content_images[0].shape)
  style_images = torch.cat(style_images).reshape(len(style_images), *style_images[0].shape)
  content_images = content_images.to(device)
  style_images = style_images.to(device)
  #adjust_learning_rate(optimizer, iteration_count=i)
  loss_c, loss_s, l_identity1, l_identity2,loss_v = network(content_images, style_images)
  loss = 1 * loss_c + loss_s * 3 + (l_identity1 * 1) + (l_identity2 * 50) #+ loss_v * 20

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if (i + 1) % 1000 == 0:
    test_image("/content/brad_pitt.jpg","/content/023.jpg")
    test_image("/content/brad_pitt.jpg","/content/la_muse.jpg")
    test_image("/content/brad_pitt.jpg","/content/impronte_d_artista.jpg")
    test_image("/content/brad_pitt.jpg","/content/fire.jpg")
    test_image("/content/brad_pitt.jpg","/content/goph.jpg")
    test_image("/content/brad_pitt.jpg","/content/in1.jpg")
    test_image("/content/brad_pitt.jpg","/content/in2.jpg")
    test_image("/content/brad_pitt.jpg","/content/in3.jpg")
    test_image("/content/brad_pitt.jpg","/content/in4.jpg")

    test_image("/content/cat.jpg","/content/023.jpg")
    test_image("/content/cat.jpg","/content/la_muse.jpg")
    test_image("/content/cat.jpg","/content/impronte_d_artista.jpg")
    test_image("/content/cat.jpg","/content/fire2.jpg")
    test_image("/content/cat.jpg","/content/goph.jpg")
    test_image("/content/cat.jpg","/content/in1.jpg")
    test_image("/content/cat.jpg","/content/in2.jpg")
    test_image("/content/cat.jpg","/content/in3.jpg")
    test_image("/content/cat.jpg","/content/in4.jpg")

  if (i + 1) % 100 == 0:
    loss_c_array.append(loss_c.data.item())
    loss_s_array.append(loss_s.data.item())
    loss_identity_1_array.append(l_identity1.data.item())
    loss_identity_2_array.append(l_identity2.data.item())
    loss_array.append(loss.data.item())
    count = len(loss_array)
    x = np.arange(count)
    plt.plot(x, loss_array, label="loss")
    plt.show()

  if (i + 1) % SAVE_MODEL_INTERVAL == 0 or (i + 1) == MAX_ITER:
        state_dict = network.decoder.state_dict()
        for key in state_dict.keys():
            state_dict[key] = state_dict[key].to(torch.device('cpu'))
        torch.save(state_dict,'/content/drive/My Drive/SANet++/decoder_iter_{}_63.pth'.format(i + 1))
        state_dict = network.sa_module.state_dict()
        for key in state_dict.keys():
            state_dict[key] = state_dict[key].to(torch.device('cpu'))
        torch.save(state_dict,'/content/drive/My Drive/SANet++/sa_module_iter_{}_63.pth'.format(i + 1))
        state_dict = network.vgg_stylized_content.state_dict()
        for key in state_dict.keys():
            state_dict[key] = state_dict[key].to(torch.device('cpu'))
        torch.save(state_dict,'/content/drive/My Drive/SANet++/vgg_stylized_content_iter_{}_63.pth'.format(i + 1))
        state_dict = network.vgg_stylized_style.state_dict()
        for key in state_dict.keys():
            state_dict[key] = state_dict[key].to(torch.device('cpu'))
        torch.save(state_dict,'/content/drive/My Drive/SANet++/vgg_stylized_style_iter_{}_63.pth'.format(i + 1))